<a href="https://colab.research.google.com/github/amydst/Data-Science-24/blob/main/Delayed_Flights_temporary_view_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from https://downloads.apache.org/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.1'
spark_version = 'spark-3.5.4'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [61.9 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,590 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Package

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DelayedFlightsView").getOrCreate()


In [3]:
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/3/DelayedFlights.csv"
spark.sparkContext.addFile(url)
delayed_flights_df = spark.read.csv(SparkFiles.get("DelayedFlights.csv"), sep=",", header=True)
delayed_flights_df.show()


+---+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
| id|Year|Month|DayofMonth|DayOfWeek|DepTime|CRSDepTime|ArrTime|CRSArrTime|UniqueCarrier|FlightNum|TailNum|ActualElapsedTime|CRSElapsedTime|AirTime|ArrDelay|DepDelay|Origin|Dest|Distance|TaxiIn|TaxiOut|Cancelled|CancellationCode|Diverted|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+---+----+-----+----------+---------+-------+----------+-------+----------+-------------+---------+-------+-----------------+--------------+-------+--------+--------+------+----+--------+------+-------+---------+----------------+--------+------------+------------+--------+-------------+-----------------+
|  0|2008|    1|         3|        4|   2003|      1955|   2211|      2225|       

In [4]:
# Create a temporary view.
delayed_flights_df.createOrReplaceTempView('delayed_view')

In [8]:
# Get the first five airline carriers and their origin that will arrive at George Bush Intercontinental Airport (IAH) in Houston.
# Hint: Pass in the airport code as a string.
spark.sql("SELECT UniqueCarrier, Origin FROM delayed_view WHERE Dest LIKE 'IAH' LIMIT 5").show()

+-------------+------+
|UniqueCarrier|Origin|
+-------------+------+
|           XE|   ABQ|
|           XE|   MCI|
|           XE|   ABQ|
|           XE|   MCI|
|           XE|   CRW|
+-------------+------+



In [11]:
# Get all the origin and destination of all the Southwest (WN) flights.

spark.sql("SELECT UniqueCarrier, Origin, Dest FROM delayed_view WHERE UniqueCarrier LIKE 'WN'").show()

+-------------+------+----+
|UniqueCarrier|Origin|Dest|
+-------------+------+----+
|           WN|   IAD| TPA|
|           WN|   IAD| TPA|
|           WN|   IND| BWI|
|           WN|   IND| BWI|
|           WN|   IND| JAX|
|           WN|   IND| LAS|
|           WN|   IND| MCO|
|           WN|   IND| MCO|
|           WN|   IND| MDW|
|           WN|   IND| PHX|
|           WN|   IND| PHX|
|           WN|   IND| TPA|
|           WN|   ISP| BWI|
|           WN|   ISP| BWI|
|           WN|   ISP| BWI|
|           WN|   ISP| BWI|
|           WN|   ISP| BWI|
|           WN|   ISP| FLL|
|           WN|   ISP| FLL|
|           WN|   ISP| MCO|
+-------------+------+----+
only showing top 20 rows



In [13]:
# Get the carrier, the origin, the destination, and the elapsed time of the 10 flights in descending order.
spark.sql("SELECT UniqueCarrier, Origin, Dest, CRSElapsedTime FROM delayed_view ORDER BY CRSElapsedTime DESC LIMIT 10").show()

+-------------+------+----+--------------+
|UniqueCarrier|Origin|Dest|CRSElapsedTime|
+-------------+------+----+--------------+
|           AS|   OAK| PDX|            99|
|           XE|   XNA| IAH|            99|
|           AS|   SFO| PDX|            99|
|           XE|   CLE| BNA|            99|
|           AS|   SFO| PDX|            99|
|           XE|   IAH| MGM|            99|
|           AS|   SFO| PDX|            99|
|           XE|   CLE| MHT|            99|
|           AS|   SFO| PDX|            99|
|           XE|   CLE| MHT|            99|
+-------------+------+----+--------------+



In [16]:
# Get the total number of diverted flights from each airline carrier, and group the results by carrier
# Order by the total number of diverted flights in descending order.

spark.sql("SELECT UniqueCarrier, SUM(Diverted) AS Total_Diverted FROM delayed_view GROUP BY UniqueCarrier ORDER BY Total_Diverted DESC").show()


+-------------+--------------+
|UniqueCarrier|Total_Diverted|
+-------------+--------------+
|           WN|         739.0|
|           AA|         469.0|
|           MQ|         370.0|
|           XE|         293.0|
|           OO|         292.0|
|           UA|         244.0|
|           DL|         196.0|
|           US|         189.0|
|           FL|         179.0|
|           9E|         155.0|
|           NW|         145.0|
|           YV|         130.0|
|           CO|         130.0|
|           AS|         125.0|
|           OH|         104.0|
|           B6|          97.0|
|           F9|          29.0|
|           AQ|           6.0|
|           HA|           4.0|
|           EV|           0.0|
+-------------+--------------+



In [17]:
# Get the average time for delayed departures and arrivals for each carrier.
# Group the results by the carrier.

spark.sql("""
    SELECT UniqueCarrier,
           AVG(DepDelay) AS Avg_DepDelay,
           AVG(ArrDelay) AS Avg_ArrDelay
    FROM delayed_view
    GROUP BY UniqueCarrier
""").show()




+-------------+------------------+------------------+
|UniqueCarrier|      Avg_DepDelay|      Avg_ArrDelay|
+-------------+------------------+------------------+
|           UA| 50.31698812513716| 49.28722883905207|
|           AA|   46.840312257564| 47.79455631216452|
|           NW| 41.70838669696344| 45.91269035532995|
|           EV| 45.39245477069796| 43.73542611378617|
|           B6| 50.69227741822634| 49.45584295814852|
|           DL|37.734883816069384| 38.02667789369917|
|           OO| 45.18429695982628| 46.39657709707309|
|           F9|  25.6334499562664|26.108155473493145|
|           YV| 58.29157351676698| 58.51809551208285|
|           US| 38.55870287528075| 35.11325534609939|
|           AQ|             26.68| 21.25537634408602|
|           MQ|  44.9969698806133| 47.34964387897973|
|           OH| 46.26107985729967| 48.05260258881851|
|           HA| 34.00154023873701|33.811415349016585|
|           XE| 49.27562001311182| 49.51955145712174|
|           AS|38.8886002537